# Лабораторная работа 4

Гуськова Кристина Игоревна, 6409-010302D

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [2]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы.

In [3]:
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

170498071/170498071 [==============================] - 2s 0us/step
Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [4]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y

        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [5]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 32, 32, 3) (64,)
1 (64, 32, 32, 3) (64,)
2 (64, 32, 32, 3) (64,)
3 (64, 32, 32, 3) (64,)
4 (64, 32, 32, 3) (64,)
5 (64, 32, 32, 3) (64,)
6 (64, 32, 32, 3) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети.

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [7]:
USE_GPU = True # устанавливаем флаг для использования GPU

# проверяем наличие GPU и выбираем устройство
if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

print('Using device: ', device) # выводим выбранное устройство

print_every = 100 # определяем константу для контроля частоты вывода информации во время обучения моделей

Using device:  /device:GPU:0


In [8]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_TwoLayerFC()

(64, 10)


/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Реализуйте трехслойную CNN для вашей задачи классификации.

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU
5. Полносвязный слой
6. Функция активации Softmax

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [9]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        initializer = tf.initializers.VarianceScaling(scale=2.0) # определяем инициализатор весов

        self.flatten = tf.keras.layers.Flatten() # создаем слой Flatten, который преобразует входные данные в одномерный массив

        # создаем сверточные слои conv1 и conv2
        self.conv1 = tf.keras.layers.Conv2D(channel_1, [5, 5], [1, 1], padding='same',
                                            kernel_initializer=initializer,
                                            activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(channel_2, [3, 3], [1, 1], padding='same',
                                            kernel_initializer=initializer,
                                            activation='relu')

        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer) # создаем полносвязный слой fc с указанным количеством классов и функцией активации softmax

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################

    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x) # пропускаем входные данные через первый сверточный слой conv1

        x = self.conv2(x) # пропускаем полученные данные через второй сверточный слой conv2

        x = self.flatten(x) # преобразовываем данные в плоский формат с помощью слоя Flatten

        scores = self.fc(x) # вычисляем значения оценок классов с помощью полносвязного слоя fc

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        return scores

In [10]:
def test_ThreeLayerConvNet():
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [11]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.

    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for

    Returns: Nothing, but prints progress during trainingn
    """
    with tf.device(device):


        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

        model = model_init_fn()
        optimizer = optimizer_init_fn()

        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

        t = 0
        for epoch in range(num_epochs):

            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()

            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:

                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)

                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)

                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)

                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1

In [12]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.9944493770599365, Accuracy: 4.6875, Val Loss: 2.924266815185547, Val Accuracy: 12.5
Iteration 100, Epoch 1, Loss: 2.2359302043914795, Accuracy: 28.589109420776367, Val Loss: 1.8716316223144531, Val Accuracy: 38.60000228881836
Iteration 200, Epoch 1, Loss: 2.0715084075927734, Accuracy: 32.26834487915039, Val Loss: 1.8199480772018433, Val Accuracy: 39.5
Iteration 300, Epoch 1, Loss: 2.002737045288086, Accuracy: 33.79360580444336, Val Loss: 1.8823935985565186, Val Accuracy: 36.5
Iteration 400, Epoch 1, Loss: 1.936016321182251, Accuracy: 35.617984771728516, Val Loss: 1.7014447450637817, Val Accuracy: 43.5
Iteration 500, Epoch 1, Loss: 1.8919559717178345, Accuracy: 36.79827880859375, Val Loss: 1.636350393295288, Val Accuracy: 43.5
Iteration 600, Epoch 1, Loss: 1.8604825735092163, Accuracy: 37.68718719482422, Val Loss: 1.645999550819397, Val Accuracy: 44.60000228881836
Iteration 700, Epoch 1, Loss: 1.8346121311187744, Accuracy: 38.40495681762695, Val Loss: 1.620

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 .

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [13]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes) # создаем модель трехслойной сверточной нейронной сети ThreeLayerConvNet

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate, 0.9, nesterov=True) # создаем оптимизатор стохастического градиентного спуска SGD с моментом и использованием Nesterov momentum

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.163867950439453, Accuracy: 3.125, Val Loss: 7.5835490226745605, Val Accuracy: 11.0
Iteration 100, Epoch 1, Loss: 2.2278711795806885, Accuracy: 25.696165084838867, Val Loss: 1.833018183708191, Val Accuracy: 35.5
Iteration 200, Epoch 1, Loss: 1.997072696685791, Accuracy: 31.786378860473633, Val Loss: 1.656759262084961, Val Accuracy: 42.29999923706055
Iteration 300, Epoch 1, Loss: 1.8762261867523193, Accuracy: 35.21075439453125, Val Loss: 1.5778638124465942, Val Accuracy: 45.400001525878906
Iteration 400, Epoch 1, Loss: 1.7847610712051392, Accuracy: 38.084476470947266, Val Loss: 1.4887930154800415, Val Accuracy: 47.900001525878906
Iteration 500, Epoch 1, Loss: 1.7244035005569458, Accuracy: 39.77046203613281, Val Loss: 1.4418889284133911, Val Accuracy: 48.89999771118164
Iteration 600, Epoch 1, Loss: 1.6797679662704468, Accuracy: 41.061771392822266, Val Loss: 1.4115409851074219, Val Accuracy: 49.599998474121094
Iteration 700, Epoch 1, Loss: 1.6424200534820557, 

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [14]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (32, 32, 3)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax',
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.6791768074035645, Accuracy: 15.625, Val Loss: 2.974302053451538, Val Accuracy: 13.600000381469727
Iteration 100, Epoch 1, Loss: 2.2370293140411377, Accuracy: 28.86757469177246, Val Loss: 1.9311957359313965, Val Accuracy: 37.70000076293945
Iteration 200, Epoch 1, Loss: 2.0839684009552, Accuracy: 32.20615768432617, Val Loss: 1.8250834941864014, Val Accuracy: 39.099998474121094
Iteration 300, Epoch 1, Loss: 2.004880666732788, Accuracy: 34.14140319824219, Val Loss: 1.8983943462371826, Val Accuracy: 36.89999771118164
Iteration 400, Epoch 1, Loss: 1.934259295463562, Accuracy: 35.86736297607422, Val Loss: 1.7530382871627808, Val Accuracy: 41.10000228881836
Iteration 500, Epoch 1, Loss: 1.889353632926941, Accuracy: 36.94797897338867, Val Loss: 1.6591806411743164, Val Accuracy: 42.5
Iteration 600, Epoch 1, Loss: 1.85896897315979, Accuracy: 37.863975524902344, Val Loss: 1.686116337776184, Val Accuracy: 42.20000076293945
Iteration 700, Epoch 1, Loss: 1.83372473716735

Альтернативный менее гибкий способ обучения:

In [15]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 9s 28ms/step - loss: 1.7622 - sparse_categorical_accuracy: 0.4013


[1.7621982097625732, 0.40130001306533813]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [16]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    input_shape = (28,28,1) # определяем форму входных данных
    channel_1, channel_2, num_classes = 28, 14, 10 # определяем параметры для создания модели нейронной сети
    initializer = tf.initializers.VarianceScaling(scale=2.0) # создаем инициализатор весов с помощью VarianceScaling

    # определяем слои модели нейронной сети
    # в данном случае используются сверточные слои, слой Flatten для преобразования входных данных в плоский формат и полносвязный слой для вывода оценок классов с функцией активации softmax
    layers = [
        tf.keras.layers.Conv2D(channel_1, [5,5], [1,1], padding='same',
                               kernel_initializer=initializer,
                               activation='relu'),
        tf.keras.layers.Conv2D(channel_2, [3,3], [1,1], padding='same',
                               kernel_initializer=initializer,
                               activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes,
                              activation='softmax',
                              kernel_initializer=initializer)
    ]
    model = tf.keras.Sequential(layers) # создаем последовательную модель Sequential с определенными слоями

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate, momentum=0.9, nesterov=True) # создаем оптимизатор стохастического градиентного спуска SGD с моментом и использованием метода Нестерова

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.0395925045013428, Accuracy: 4.6875, Val Loss: 2.8093976974487305, Val Accuracy: 8.0
Iteration 100, Epoch 1, Loss: 2.087306499481201, Accuracy: 26.48514747619629, Val Loss: 1.839172124862671, Val Accuracy: 35.400001525878906
Iteration 200, Epoch 1, Loss: 1.9420149326324463, Accuracy: 31.397701263427734, Val Loss: 1.6988357305526733, Val Accuracy: 42.39999771118164
Iteration 300, Epoch 1, Loss: 1.8559588193893433, Accuracy: 34.60859680175781, Val Loss: 1.6580696105957031, Val Accuracy: 43.29999923706055
Iteration 400, Epoch 1, Loss: 1.7893253564834595, Accuracy: 36.99345397949219, Val Loss: 1.6093735694885254, Val Accuracy: 44.70000076293945
Iteration 500, Epoch 1, Loss: 1.742525339126587, Accuracy: 38.541664123535156, Val Loss: 1.549770712852478, Val Accuracy: 46.20000076293945
Iteration 600, Epoch 1, Loss: 1.7108122110366821, Accuracy: 39.72025680541992, Val Loss: 1.5107885599136353, Val Accuracy: 47.60000228881836
Iteration 700, Epoch 1, Loss: 1.683397531

In [17]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 7s 21ms/step - loss: 1.4499 - sparse_categorical_accuracy: 0.4771


[1.4498828649520874, 0.4771000146865845]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры.

Ниже представлен пример для полносвязной сети.

In [18]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)

    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)

    with tf.device(device):
        scores = model(x)
        print(scores.shape)

test_two_layer_fc_functional()

(64, 10)


In [19]:
input_shape = (32, 32, 3)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.2285590171813965, Accuracy: 7.8125, Val Loss: 3.079054117202759, Val Accuracy: 11.59999942779541
Iteration 100, Epoch 1, Loss: 2.240535259246826, Accuracy: 28.171411514282227, Val Loss: 1.876623272895813, Val Accuracy: 39.39999771118164
Iteration 200, Epoch 1, Loss: 2.0787038803100586, Accuracy: 32.019588470458984, Val Loss: 1.8760384321212769, Val Accuracy: 38.5
Iteration 300, Epoch 1, Loss: 2.003521203994751, Accuracy: 34.04277420043945, Val Loss: 1.8371996879577637, Val Accuracy: 37.0
Iteration 400, Epoch 1, Loss: 1.9341065883636475, Accuracy: 35.925811767578125, Val Loss: 1.7063727378845215, Val Accuracy: 41.5
Iteration 500, Epoch 1, Loss: 1.8896411657333374, Accuracy: 36.907432556152344, Val Loss: 1.656093716621399, Val Accuracy: 43.599998474121094
Iteration 600, Epoch 1, Loss: 1.8584963083267212, Accuracy: 37.85097885131836, Val Loss: 1.676419734954834, Val Accuracy: 43.0
Iteration 700, Epoch 1, Loss: 1.8325554132461548, Accuracy: 38.51640701293945, 

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут).

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [24]:
class CustomConvNet(tf.keras.Model):
    def __init__(self):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.conv1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3)) # определяем сверточный слой conv1 с 16 фильтрами, размером ядра (3, 3) и функцией активации ReLU

        self.conv2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu') # определяем сверточный слой conv2 с 32 фильтрами, размером ядра (3, 3) и функцией активации ReLU

        self.pool1 = tf.keras.layers.MaxPooling2D((2, 2)) # определяем слой пулинга pool1 с размером окна (2, 2)

        self.dropout1 = tf.keras.layers.Dropout(0.25) # определяем слой dropout1 с коэффициентом отсева 0.25

        self.conv3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu') # определяем сверточный слой conv3 с 64 фильтрами, размером ядра (3, 3) и функцией активации ReLU

        self.flatten = tf.keras.layers.Flatten() # определяем слой flatten для преобразования данных в одномерный массив

        self.dense1 = tf.keras.layers.Dense(128, activation='relu') # определяем полносвязный слой dense1 с 128 нейронами и функцией активации ReLU

        self.dropout2 = tf.keras.layers.Dropout(0.4) # определяем слой dropout2 с коэффициентом отсева 0.4

        self.dense2 = tf.keras.layers.Dense(10, activation='softmax') # определяем полносвязный слой dense2 с 10 нейронами (по числу классов) и функцией активации softmax

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################

    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(input_tensor) # применяем сверточный слой conv1 к входному тензору input_tensor
        x = self.conv2(x) # применяем сверточный слой conv2 к выходу из предыдущего слоя
        x = self.pool1(x) # применяем слой пулинга pool1 к выходу из сверточного слоя conv2

        # если режим обучения равен True, то применяется слой dropout1 к выходу из слоя pool1
        if training:
          x = self.dropout1(x, training=training)

        x = self.conv3(x) # применяем сверточный слой conv3 к выходу из слоя dropout1 или pool1, если dropout1 не был применен
        x = self.flatten(x) # преобразовываем выход из сверточного слоя conv3 в одномерный массив
        x = self.dense1(x) # применяем полносвязный слой dense1 к выходу из слоя flatten

        # если режим обучения равен True, то применяется слой dropout2 к выходу из слоя dense1
        if training:
          x = self.dropout2(x, training=training)

        x = self.dense2(x) # применяем полносвязный слой dense2 к выходу из слоя dropout2 или dense1, если dropout2 не был применен

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 300
num_epochs = 10

model = CustomConvNet()

def model_init_fn():
    return CustomConvNet()

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.302915096282959, Accuracy: 12.5, Val Loss: 2.4385130405426025, Val Accuracy: 7.90000057220459
Iteration 300, Epoch 1, Loss: 1.7545247077941895, Accuracy: 36.38911819458008, Val Loss: 1.413498044013977, Val Accuracy: 52.0
Iteration 600, Epoch 1, Loss: 1.5641472339630127, Accuracy: 43.74480056762695, Val Loss: 1.1681092977523804, Val Accuracy: 59.60000228881836
Iteration 900, Epoch 2, Loss: 1.180650234222412, Accuracy: 58.344905853271484, Val Loss: 1.034635066986084, Val Accuracy: 65.20000457763672
Iteration 1200, Epoch 2, Loss: 1.1376639604568481, Accuracy: 59.93893814086914, Val Loss: 1.0103025436401367, Val Accuracy: 65.69999694824219
Iteration 1500, Epoch 2, Loss: 1.1100674867630005, Accuracy: 60.93962860107422, Val Loss: 0.9688239097595215, Val Accuracy: 67.5999984741211
Iteration 1800, Epoch 3, Loss: 1.0023555755615234, Accuracy: 65.22421264648438, Val Loss: 0.9039028286933899, Val Accuracy: 69.19999694824219
Iteration 2100, Epoch 3, Loss: 0.9777933955

Опишите все эксперименты, результаты. Сделайте выводы.

Архитектура сети состоит из: сверточных слоев, максимального пулинга, dropout, flatten и полносвязных слоев.

На одинаковых настройках были протестированы различные оптимизаторы: SGD, Adam, RMSprop. Первый показал наихудшие результаты - за 10 эпох точность была в районе 33%. Последние два показали примерно одинаковые результаты, но в большинстве своем Adam был на несколько сотых точнее.

На входном и скрытом слоях сравнивались функции активации relu и sigmoid, а на входном слое тестировались sigmoid и softmax. При комбинации sigmoid, softmax и Adam был получен наихудший результат - для получения необходимой точности нужно было примерно 15 эпох. При комбинации relu, softmax и Adam был получен наилучший результат - 73% accuracy на валидационной выборке за 10 эпох обучения.